In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googleapis/langchain-google-alloydb-pg-python/blob/main/samples/langchain_quick_start.ipynb)

---
# Introduction

In this codelab, you'll learn how to create a powerful interactive generative AI application using Retrieval Augmented Generation powered by [AlloyDB for PostgreSQL](https://cloud.google.com/alloydb) and [LangChain](https://www.langchain.com/). We will be creating an application grounded in a [Netflix Movie dataset](https://www.kaggle.com/datasets/shivamb/netflix-shows), allowing you to interact with movie data in exciting new ways.

## Prerequisites

* A basic understanding of the Google Cloud Console
* Basic skills in command line interface and Google Cloud shell
* Basic python knowledge

## What you'll learn

* How to deploy a AlloyDB for PostgreSQL instance
* How to use AlloyDB for PostgreSQL as a VectorStore
* How to use AlloyDB for PostgreSQL as a DocumentLoader
* How to use AlloyDB for PostgreSQL for ChatHistory storage

## What you'll need

* A Google Cloud Account and Google Cloud Project
* A web browser such as [Chrome](https://www.google.com/chrome/)

# Setup and Requirements

In the following instructions you will learn to:

1. Install required dependencies for our application
2. Set up authentication for our project
3. Set up a AlloyDB for PostgreSQL Instance
4. Import the data used by our application

## Install dependencies
First you will need to install the dependencies needed to run this demo app.

In [19]:
%pip install langchain-google-alloydb-pg langchain langchain-google-vertexai google-cloud-alloydb-connector[pg8000]

## Authenticate to Google Cloud within Colab
In order to access your Google Cloud Project from this notebook, you will need to Authenticate as an IAM user.

In [18]:
from google.colab import auth

auth.authenticate_user()

## Connect Your Google Cloud Project

In [ ]:
# @markdown Please fill in the value below with your GCP project ID and then run the cell.

# Please fill in these values.
project_id = "rag-alloydb-poc"  # @param {type:"string"}

# Quick input validations.
assert project_id, "⚠️ Please provide a Google Cloud project ID"

# Configure gcloud.
!gcloud config set project {project_id}

## Enable APIs for AlloyDB and Vertex AI within your project

You will need to enable these APIs in order to create an AlloyDB database and utilize Vertex AI as an embeddings service!

In [ ]:
# enable GCP services
!gcloud services enable alloydb.googleapis.com aiplatform.googleapis.com

## Set up AlloyDB
You will need a Postgres AlloyDB instance for the following stages of this notebook. Please set the following variables.

In [22]:
# @markdown Please fill in the both the Google Cloud region and name of your AlloyDB instance. Once filled in, run the cell.

# Please fill in these values.
region = "us-central1"  # @param {type:"string"}
cluster_name = "my-alloydb-cluster"  # @param {type:"string"}
instance_name = "my-primary"  # @param {type:"string"}
database_name = "langchain"  # @param {type:"string"}
password = input("Please provide a password to be used for 'postgres' database user: ")

Please provide a password to be used for 'postgres' database user: postgres


### Create an AlloyDB Instance
If you have already created an AlloyDB Cluster and Instance, you can skip these steps and skip to the connectivity section.

> ⏳ - Creating an AlloyDB cluster may take a few minutes.

In [23]:
# Quick input validations.
assert region, "⚠️ Please provide a Google Cloud region"
assert instance_name, "⚠️ Please provide the name of your instance"
assert database_name, "⚠️ Please provide the name of your database_name"

# create the AlloyDB Cluster
!gcloud beta alloydb clusters create {cluster_name} --password={password} --region={region}

Operation ID: operation-1737886592674-62c99410429b4-d32d7559-226a15cd


Create an instance attached to our cluster with the following command.
> ⏳ - Creating an AlloyDB instance may take a few minutes.

In [24]:
!gcloud beta alloydb instances create {instance_name} --instance-type=PRIMARY --cpu-count=2 --region={region} --cluster={cluster_name}

Operation ID: operation-1737886607471-62c9941e5f369-eb419ced-efc72046


To connect to your AlloyDB instance from this notebook, you will need to enable public IP on your instance. Alternatively, you can follow [these instructions](https://cloud.google.com/alloydb/docs/connect-external) to connect to an AlloyDB for PostgreSQL instance with Private IP from outside your VPC.

In [25]:
!gcloud beta alloydb instances update {instance_name} --region={region} --cluster={cluster_name} --assign-inbound-public-ip=ASSIGN_IPV4
#Enable public ID of from UI

ERROR: (gcloud.beta.alloydb.instances.update) INVALID_ARGUMENT: The request was invalid: password complexity flag password.enforce_complexity is required when public IP is enabled
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - field: database_flags
- '@type': type.googleapis.com/google.rpc.RequestInfo
  requestId: 7ee4e55e0f3027da


Now create a connection pool to connect to our instance.

In [26]:
from google.cloud.alloydb.connector import Connector, IPTypes
import sqlalchemy


connection_string = f"projects/{project_id}/locations/{region}/clusters/{cluster_name}/instances/{instance_name}"
# initialize Connector object
connector = Connector()


# function to return the database connection
def getconn():
    conn = connector.connect(
        connection_string,
        "pg8000",
        user="postgres",
        password=password,
        db="postgres",
        enable_iam_auth=False,
        ip_type=IPTypes.PUBLIC,
    )
    return conn


# create connection pool
pool = sqlalchemy.create_engine(
    "postgresql+pg8000://", creator=getconn, isolation_level="AUTOCOMMIT"
)

### Create a Database

Next you will create database to store the data for this application using the connection pool. Enabling public IP takes a few minutes, you may get an error that there is no public IP address. Please wait and retry this step if you hit an error!

In [27]:
with pool.connect() as db_conn:
    db_conn.execute(sqlalchemy.text(f"CREATE DATABASE {database_name}"))
connector.close()

#### Connect to our New Database

Now you will add in a connection function that connects to your new database!

In [28]:
from google.cloud.alloydb.connector import Connector, IPTypes
import sqlalchemy


connection_string = f"projects/{project_id}/locations/{region}/clusters/{cluster_name}/instances/{instance_name}"
# initialize Connector object
connector = Connector()


# function to return the database connection
def getconn():
    conn = connector.connect(
        connection_string,
        "pg8000",
        user="postgres",
        password=password,
        db=database_name,
        enable_iam_auth=False,
        ip_type=IPTypes.PUBLIC,
    )
    return conn


# create connection pool
pool = sqlalchemy.create_engine("postgresql+pg8000://", creator=getconn)

### Import data to your database

Now that you have your database, you will need to import data! We will be using a [Netflix Dataset from Kaggle](https://www.kaggle.com/datasets/shivamb/netflix-shows). Here is what the data looks like:

| show_id | type    | title                | director         | cast                                                                                                                                                  | country       | date_added        | release_year | rating | duration  | listed_in                                    | description                                                                                                                                                                           |
|---------|---------|----------------------|------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------|---------------|-------------------|--------------|--------|-----------|----------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| s1      | Movie   | Dick Johnson Is Dead | Kirsten Johnson  |                                                                                                                                                        | United States | September 25, 2021 | 2020         | PG-13  | 90 min    | Documentaries                                | As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable.                              |
| s2      | TV Show | Blood & Water        |                  | Ama Qamata, Khosi Ngema, Gail Mabalane, Thabang Molaba, Dillon Windvogel, Natasha Thahane, Arno Greeff, Xolile Tshabalala, Getmore Sithole, Cindy Mahlangu, Ryle De Morny, Greteli Fincham, Sello Maake Ka-Ncube, Odwa Gwanya, Mekaila Mathys, Sandi Schultz, Duane Williams, Shamilla Miller, Patrick Mofokeng | South Africa  | September 24, 2021 | 2021         | TV-MA  | 2 Seasons | International TV Shows, TV Dramas, TV Mysteries | After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth.                                   |
| s3      | TV Show | Ganglands            | Julien Leclercq  | Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabiha Akkari, Sofia Lesaffre, Salim Kechiouche, Noureddine Farihi, Geert Van Rampelberg, Bakary Diombera                                   |               | September 24, 2021 | 2021         | TV-MA  | 1 Season  | Crime TV Shows, International TV Shows, TV Action & Adventure | To protect his family from a powerful drug lord, skilled thief Mehdi and his expert team of robbers are pulled into a violent and deadly turf war.                                     |
| s4      | TV Show | Jailbirds New Orleans |                  |                                                                                                                                                        |               | September 24, 2021 | 2021         | TV-MA  | 1 Season  | Docuseries, Reality TV                        | Feuds, flirtations and toilet talk go down among the incarcerated women at the Orleans Justice Center in New Orleans on this gritty reality series.                                   |
| s5      | TV Show | Kota Factory         |                  | Mayur More, Jitendra Kumar, Ranjan Raj, Alam Khan, Ahsaas Channa, Revathi Pillai, Urvi Singh, Arun Kumar                                                 | India        | September 24, 2021 | 2021         | TV-MA  | 2 Seasons | International TV Shows, Romantic TV Shows, TV Comedies | In a city of coaching centers known to train India’s finest collegiate minds, an earnest but unexceptional student and his friends navigate campus life. |


The following code has been prepared code to help insert the CSV data into your AlloyDB for PostgreSQL database.

Download the CSV file:

In [29]:
!gsutil cp gs://cloud-samples-data/langchain/common/first_five_netflix_titles.csv .

Copying gs://cloud-samples-data/langchain/common/first_five_netflix_titles.csv...
/ [1 files][  2.0 KiB/  2.0 KiB]                                                
Operation completed over 1 objects/2.0 KiB.                                      


The download can be verified by the following command or using the "Files" tab.

In [30]:
!ls

first_five_netflix_titles.csv  sample_data


In this next step you will:

1. Create the table into store data
2. And insert the data from the CSV into the database table

In [31]:
import pandas as pd

create_table_cmd = sqlalchemy.text(
    'CREATE TABLE netflix_titles ( \
    show_id VARCHAR, \
    type VARCHAR, \
    title VARCHAR, \
    director VARCHAR, \
    "cast" VARCHAR, \
    country VARCHAR, \
    date_added VARCHAR, \
    release_year INTEGER, \
    rating VARCHAR, \
    duration VARCHAR, \
    listed_in VARCHAR, \
    description TEXT \
    )',
)

netflix_data = "/content/first_five_netflix_titles.csv"

df = pd.read_csv(netflix_data)
insert_data_cmd = sqlalchemy.text(
    """
    INSERT INTO netflix_titles VALUES (:show_id, :type, :title, :director,
      :cast, :country, :date_added, :release_year, :rating,
      :duration, :listed_in, :description)
    """
)

parameter_map = [
    {
        "show_id": row["show_id"],
        "type": row["type"],
        "title": row["title"],
        "director": row["director"],
        "cast": row["cast"],
        "country": row["country"],
        "date_added": row["date_added"],
        "release_year": row["release_year"],
        "rating": row["rating"],
        "duration": row["duration"],
        "listed_in": row["listed_in"],
        "description": row["description"],
    }
    for index, row in df.iterrows()
]

with pool.connect() as db_conn:
    db_conn.execute(create_table_cmd)
    db_conn.execute(
        insert_data_cmd,
        parameter_map,
    )
    db_conn.commit()
connector.close()

## Use case 1: AlloyDB for Postgres as a document loader

---



Now that you have data in your database, you are ready to use AlloyDB for PostgreSQL as a document loader. This means you will pull data from the database and load it into memory as documents. These documents can be used to create a vector store.

First, create a connection to your AlloyDB for PostgreSQL instance using the `AlloyDBEngine` class.

In [32]:
from langchain_google_alloydb_pg import AlloyDBEngine, Column, AlloyDBLoader

engine = AlloyDBEngine.from_instance(
    project_id=project_id,
    instance=instance_name,
    region=region,
    cluster=cluster_name,
    database=database_name,
    user="postgres",
    password=password,
)

The `AlloyDBLoader` requires an `AlloyDBEngine` object to define the database connection and a `query` to define which data is to be retrieved. The `content_columns` argument can be used to define the columns that will be used as "content" in the document object we will later construct. The rest of the columns in that table will become the "metadata" associated with the documents.

In [33]:
table_name = "netflix_titles"
content_columns = ["title", "director", "cast", "description"]
loader = await AlloyDBLoader.create(
    engine=engine,
    query=f"SELECT * FROM {table_name};",
    content_columns=content_columns,
)

Use method `aload()` to pull documents from out database. You can see the first 5 documents from the database here.

In [34]:
documents = await loader.aload()
print(f"Loaded {len(documents)} from the database. 5 Examples:")
for doc in documents[:5]:
    print(doc)

Loaded 5 from the database. 5 Examples:
page_content='Dick Johnson Is Dead Kirsten Johnson nan As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable.' metadata={'show_id': 's1', 'type': 'Movie', 'country': 'United States', 'date_added': 'September 25, 2021', 'release_year': 2020, 'rating': 'PG-13', 'duration': '90 min', 'listed_in': 'Documentaries'}
page_content='Blood & Water nan Ama Qamata, Khosi Ngema, Gail Mabalane, Thabang Molaba, Dillon Windvogel, Natasha Thahane, Arno Greeff, Xolile Tshabalala, Getmore Sithole, Cindy Mahlangu, Ryle De Morny, Greteli Fincham, Sello Maake Ka-Ncube, Odwa Gwanya, Mekaila Mathys, Sandi Schultz, Duane Williams, Shamilla Miller, Patrick Mofokeng After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth.' metadata={'show_id': 's2', 'type': 'TV Show', 'country': 'South A

Nice, you just used AlloyDB for Postgres as a document loader!

## Use case 2: AlloyDB for PostgreSQL as Vector Store

Now, you will learn how to put all of the documents into a vector store so that you perform a vector search.

### Create Your Vector Store table

Create a vector store table that can preserve the Document's metadata by using the method `init_vectorstore_table` and defining specific metadata columns. The vector size is required. The example shows the vector size, `768`, that corresponds with the length of the vectors computed by the model our embeddings service uses, Vertex AI's `textembedding-gecko`.

In [35]:
from langchain_google_alloydb_pg import AlloyDBEngine, Column

sample_vector_table_name = "movie_vector_table_samples"

engine = AlloyDBEngine.from_instance(
    project_id=project_id,
    instance=instance_name,
    region=region,
    cluster=cluster_name,
    database=database_name,
    user="postgres",
    password=password,
)

engine.init_vectorstore_table(
    sample_vector_table_name,
    vector_size=768,
    metadata_columns=[
        Column("show_id", "VARCHAR", nullable=True),
        Column("type", "VARCHAR", nullable=True),
        Column("country", "VARCHAR", nullable=True),
        Column("date_added", "VARCHAR", nullable=True),
        Column("release_year", "INTEGER", nullable=True),
        Column("rating", "VARCHAR", nullable=True),
        Column("duration", "VARCHAR", nullable=True),
        Column("listed_in", "VARCHAR", nullable=True),
    ],
    overwrite_existing=True,  # Enabling this will recreate the table if exists.
)

### Try inserting the documents into the vector table

Next, you will create a `AlloyDBVectorStore` object that connects to the new AlloyDB database table to store the data from the documents. Note that for each row, the embedding service will be called to compute the embeddings to store in the vector table. Pricing details can be found [here](https://cloud.google.com/vertex-ai/pricing).

In [36]:
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_alloydb_pg import AlloyDBVectorStore, AlloyDBEngine

# Initialize the embedding service. In this case we are using version 003 of Vertex AI's textembedding-gecko model. In general, it is good practice to specify the model version used.
embeddings_service = VertexAIEmbeddings(
    model_name="textembedding-gecko@003", project=project_id
)

engine = AlloyDBEngine.from_instance(
    project_id=project_id,
    instance=instance_name,
    region=region,
    cluster=cluster_name,
    database=database_name,
    user="postgres",
    password=password,
)

vector_store = AlloyDBVectorStore.create_sync(
    engine=engine,
    embedding_service=embeddings_service,
    table_name=sample_vector_table_name,
    metadata_columns=[
        "show_id",
        "type",
        "country",
        "date_added",
        "release_year",
        "duration",
        "listed_in",
    ],
)

Now, add the documents data into the vector table. Here is a code example to load the first 5 documents in the list.

In [37]:
import uuid

docs_to_load = documents[:5]

# ! Uncomment the following line to load all 8,800+ documents to the database vector table with calling the embedding service.
# docs_to_load = documents

ids = [str(uuid.uuid4()) for i in range(len(docs_to_load))]
vector_store.add_documents(docs_to_load, ids)

['e8afeab8-d29b-4539-af67-b7c4da37b9bf',
 'dd8943e2-aeac-4bff-98fe-ba7f58017e95',
 'bf5a9b0d-f7dc-46cf-87b9-b31bc291e1d5',
 '2a80e2da-f1e3-4834-b590-e8e360dea628',
 '1aada704-cb7c-4c3c-aa6e-a2683d0789be']

### Import the rest of your data into your vector table

You don't have to call the embedding service 8,800 times to load all the documents for the demo. Instead, we have prepared a CSV with the all 8,800+ rows with pre-computed embeddings in a CSV file. You can import the CSV using `gsutil`.

In [38]:
!gsutil cp gs://cloud-samples-data/langchain/alloydb/netflix_titles_embeddings.csv .

Copying gs://cloud-samples-data/langchain/alloydb/netflix_titles_embeddings.csv...
- [1 files][ 83.6 MiB/ 83.6 MiB]                                                
Operation completed over 1 objects/83.6 MiB.                                     


Use the following code to insert the pregenerated embeddings into your vector store.

In [39]:
netflix_data = "/content/netflix_titles_embeddings.csv"
df = pd.read_csv(netflix_data)
insert_data_cmd = sqlalchemy.text(
    """
    INSERT INTO movie_vector_table_samples VALUES (:langchain_id, :content, :embedding, :show_id,
      :type, :country, :date_added, :release_year, :rating,
      :duration, :listed_in, :langchain_metadata)
    """
)

parameter_map = [
    {
        "langchain_id": row["langchain_id"],
        "content": row["content"],
        "embedding": row["embedding"],
        "show_id": row["show_id"],
        "type": row["type"],
        "country": row["country"],
        "date_added": row["date_added"],
        "release_year": row["release_year"],
        "rating": row["rating"],
        "duration": row["duration"],
        "listed_in": row["listed_in"],
        "langchain_metadata": row["langchain_metadata"],
    }
    for index, row in df.iterrows()
]

with pool.connect() as db_conn:
    db_conn.execute(
        insert_data_cmd,
        parameter_map,
    )
    db_conn.commit()
connector.close()

# Use case 3: AlloyDB for PostgreSQL as Chat Memory

Next you will add chat history (called “memory” in the context of LangChain) to our application so the LLM can retain context and information across multiple interactions, leading to more coherent and sophisticated conversations or text generation. You can use AlloyDB for PostgreSQL as “memory” storage in our application so that the LLM can use context from prior conversations to better answer the user’s prompts.

In [40]:
from langchain_google_alloydb_pg import AlloyDBChatMessageHistory, AlloyDBEngine

engine = AlloyDBEngine.from_instance(
    project_id=project_id,
    instance=instance_name,
    region=region,
    cluster=cluster_name,
    database=database_name,
    user="postgres",
    password=password,
)
message_table_name = "message_store"

engine.init_chat_history_table(table_name=message_table_name)

chat_history = AlloyDBChatMessageHistory.create_sync(
    engine,
    session_id="my-test-session",
    table_name=message_table_name,
)

Here is an example of how you would add a user message and how you would add an AI message.

In [41]:
chat_history.add_user_message("Hi!")
chat_history.add_ai_message("Hello there. I'm a model and am happy to help!")

chat_history.messages

[HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hello there. I'm a model and am happy to help!", additional_kwargs={}, response_metadata={})]

## Conversational RAG Chain backed by AlloyDB

Try using all 3 integrations with the `ConversationalRetrievalChain`.

You will build a chatbot that can answer movie related questions based on the vector search results.

In [42]:
from langchain_google_vertexai import VertexAIEmbeddings, VertexAI
from langchain_core.messages import AIMessage, HumanMessage
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationSummaryBufferMemory
from langchain_core.prompts import PromptTemplate
from langchain_google_alloydb_pg import (
    AlloyDBEngine,
    AlloyDBVectorStore,
    AlloyDBChatMessageHistory,
)

# Intialize the embedding service
embeddings_service = VertexAIEmbeddings(
    model_name="textembedding-gecko@003", project=project_id
)

# Intialize the engine
engine = AlloyDBEngine.from_instance(
    project_id=project_id,
    instance=instance_name,
    region=region,
    cluster=cluster_name,
    database=database_name,
    user="postgres",
    password=password,
)
# Intialize the Vector Store
vector_store = AlloyDBVectorStore.create_sync(
    engine=engine,
    embedding_service=embeddings_service,
    table_name=sample_vector_table_name,
    metadata_columns=[
        "show_id",
        "type",
        "country",
        "date_added",
        "release_year",
        "duration",
        "listed_in",
    ],
)

# Intialize the AlloyDBChatMessageHistory
chat_history = AlloyDBChatMessageHistory.create_sync(
    engine,
    session_id="my-test-session",
    table_name="message_store",
)

Create a prompt for the LLM. Here we can add instructions specific to our application, such as "Don't make things up".

In [43]:
# Prepare some prompt templates for the ConversationalRetrievalChain
prompt = PromptTemplate(
    template="""Use all the information from the context and the conversation history to answer new question. If you see the answer in previous conversation history or the context. \
Answer it with clarifying the source information. If you don't see it in the context or the chat history, just say you \
didn't find the answer in the given data. Don't make things up.

Previous conversation history from the questioner. "Human" was the user who's asking the new question. "Assistant" was you as the assistant:
```{chat_history}
```

Vector search result of the new question:
```{context}
```

New Question:
```{question}```

Answer:""",
    input_variables=["context", "question", "chat_history"],
)
condense_question_prompt_passthrough = PromptTemplate(
    template="""Repeat the following question:
{question}
""",
    input_variables=["question"],
)

Next, create a retriever from the vector store in order to retrieve similar documents via a vector search.

In [44]:
# Initialize retriever, llm and memory for the chain
retriever = vector_store.as_retriever(
    search_type="mmr", search_kwargs={"k": 5, "lambda_mult": 0.8}
)

Next, initialize our LLM, in this case we are using Vertex AI's "gemini-pro".

In [45]:
llm = VertexAI(model_name="gemini-pro", project=project_id)

Clear your chat history, so that our application starts without any prior context to other conversations we have had with the application.

In [46]:
chat_history.clear()

memory = ConversationSummaryBufferMemory(
    llm=llm,
    chat_memory=chat_history,
    output_key="answer",
    memory_key="chat_history",
    return_messages=True,
)

<ipython-input-46-b6e34cd98860>:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(


Now, create a conversational retrieval chain. This will allow the LLM to use chat history in it's responses, meaning we can ask it follow up questions to our questions instead of having to start from scratch after each inquiry.

In [47]:
# create the ConversationalRetrievalChain
rag_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    verbose=False,
    memory=memory,
    condense_question_prompt=condense_question_prompt_passthrough,
    combine_docs_chain_kwargs={"prompt": prompt},
)

# ask some questions
q = "What movie was Brad Pitt in?"
ans = rag_chain({"question": q, "chat_history": chat_history})["answer"]
print(f"Question: {q}\nAnswer: {ans}\n")

q = "How about Jonny Depp?"
ans = rag_chain({"question": q, "chat_history": chat_history})["answer"]
print(f"Question: {q}\nAnswer: {ans}\n")

q = "Are there movies about animals?"
ans = rag_chain({"question": q, "chat_history": chat_history})["answer"]
print(f"Question: {q}\nAnswer: {ans}\n")

# browser the chat history
chat_history.messages

<ipython-input-47-8710e06fe420>:13: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ans = rag_chain({"question": q, "chat_history": chat_history})["answer"]


Question: What movie was Brad Pitt in?
Answer: Brad Pitt has starred in numerous movies, some of which include:

- **Inglourious Basterds** (2009), a World War II film directed by Quentin Tarantino.
- **By the Sea** (2015), a drama film directed by Angelina Jolie.
- **Killing Them Softly** (2012), a crime thriller film directed by Andrew Dominik.
- **Babel** (2006), a drama film directed by Alejandro G. Iñárritu.


Question: How about Jonny Depp?
Answer: ## Johnny Depp Movies:

Based on the information you provided, here are some movies starring Johnny Depp:

* **Charlie and the Chocolate Factory (2005):** A whimsical adaptation of Roald Dahl's classic children's book, where Depp plays the eccentric chocolatier Willy Wonka. 
* **The Rum Diary (2011):** Based on Hunter S. Thompson's novel, Depp portrays a journalist who finds himself caught in the chaos of Puerto Rico during the 1960s.
* **The Imaginarium of Doctor Parnassus (2009):** This fantasy film features Depp alongside Heath Ledg

[HumanMessage(content='What movie was Brad Pitt in?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Brad Pitt has starred in numerous movies, some of which include:\n\n- **Inglourious Basterds** (2009), a World War II film directed by Quentin Tarantino.\n- **By the Sea** (2015), a drama film directed by Angelina Jolie.\n- **Killing Them Softly** (2012), a crime thriller film directed by Andrew Dominik.\n- **Babel** (2006), a drama film directed by Alejandro G. Iñárritu.\n', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='How about Jonny Depp?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="## Johnny Depp Movies:\n\nBased on the information you provided, here are some movies starring Johnny Depp:\n\n* **Charlie and the Chocolate Factory (2005):** A whimsical adaptation of Roald Dahl's classic children's book, where Depp plays the eccentric chocolatier Willy Wonka. \n* **The Rum Diary (2011):** Based on Hunter S. Thompson's novel,